# Damatura, Nigeria IDP Analysis
##### January 2025

In [1]:
# standard library imports
import os
import sys

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# third-party imports
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd

# change to project root directory
os.chdir('../')
print("directory changes")
print(f"current working directory is:", os.getcwd())

# local imports (after changing directory)
from src.population import (
    clip_raster_with_shapefile,
    raster_to_points
)

from src.idp import kmz_to_shapefile, transform_idp_description, transform_idp_complete

directory changes
current working directory is: /Users/carolinecullinan/dev/wb/damaturu-nigeria-idp


1.0 Population Density

In [2]:
# check coordinate system of population density data for Nigeria, nga_pd__2020_1km_UNadj.tif
with rasterio.open("data/raw/nga_pd_2020_1km_UNadj.tif") as src:
    print(f"coordinate system: {src.crs}")

coordinate system: EPSG:4326


In [2]:
# check coordinate system of aoi, damaturu.shp
aoi = gpd.read_file("data/raw/AOI/damaturu.shp")
print(f"coordinate system: {aoi.crs}")

coordinate system: EPSG:4326


In [2]:
# apply "clip_raster_with_shapefile" function to get population density (raster) 
## of Damaturu, Nigeria specifically (shapefile) and sav interim "damaturu_pd" (raster) tif format
clip_raster_with_shapefile(
    "data/raw/nga_pd_2020_1km_UNadj.tif",
    "data/raw/AOI/damaturu.shp",
    "data/interim/damaturu_pd.tif"
)

In [3]:
# gain insight into raster dimensions to best choose a sample rate for raster to point conversion
## (i.e., determine how many points to sample from the raster, 
## where sample_rate = 1 means that every pixel from the raster is converted to a point)
with rasterio.open("data/interim/damaturu_pd.tif") as src:
    print(f"raster dimensions: {src.width} x {src.height}")
    print(f"total pixels: {src.width * src.height}")
    
# note: raster dimensions and total pixels are small, therefore use sample_rate = 1

raster dimensions: 17 x 13
total pixels: 221


In [8]:
with rasterio.open("data/interim/damaturu_pd.tif") as src:
    total_pixels = src.width * src.height
    target_points = 20000  # adjust based on your needs (web viz, aim for 10,000-50,000 points)
    suggested_sample_rate = int(total_pixels / target_points)
    print(f"suggested sample rate: {suggested_sample_rate}")

suggested sample rate: 0


In [4]:
# convert interim (raster) "damaturu_pd" to processed "damaturu_pd_points" points for export and viz
raster_to_points(
    "data/interim/damaturu_pd.tif",
    'data/processed/damaturu_pd_points.geojson',
    sample_rate=1
)

input raster dimensions: 17 x 13
total pixels: 221

Created 221 points
Value range: -99999.00 to 3183.82


,population_density,geometry
0,-99999.0,POINT (11.91125 11.79542)
1,-99999.0,POINT (11.91958 11.79542)
2,-99999.0,POINT (11.92792 11.79542)
3,-99999.0,POINT (11.93625 11.79542)
4,-99999.0,POINT (11.94458 11.79542)
...,...,...
216,-99999.0,POINT (12.01125 11.69542)
217,-99999.0,POINT (12.01958 11.69542)
218,-99999.0,POINT (12.02792 11.69542)
219,-99999.0,POINT (12.03625 11.69542)


2.0 Economic Hotspots

In [8]:
import sys
print(sys.executable)

/opt/anaconda3/bin/python


In [4]:
#nighttime light data
import ee

3.0 IDP (Location, Location by Site Type, and Change in IDP Locations by Site Types (i.e, "IDP Growth"))

In [4]:
# convert IDP location KMZ to shapefile
idp_shapefile = kmz_to_shapefile(
    "data/raw/Round_39_North_East_IDP_Camp_Sites_KMZ.kmz",
    "data/interim/idp_shapefile.shp"
)

/Users/carolinecullinan/dev/wb/damaturu-nigeria-idp/src/idp.py:31: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp)


###### 3.2 Check Shapefile Data CRS

In [3]:
# check coordinate system of IDP shapefile
print(f"coordinate system: {idp_shapefile.crs}")

coordinate system: EPSG:4326


In [5]:
# clip IDP shapefile to Damaturu
damaturu_idp = gpd.clip(idp_shapefile, aoi)

In [9]:
# save damaturu_idp to shapefile
damaturu_idp.to_file("data/interim/damaturu_idp.shp")

/var/folders/xy/9wgvl7914cj_8zq0xmgxg4hw0000gn/T/ipykernel_86795/3013792111.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  damaturu_idp.to_file("data/processed/damaturu_idp.shp")


In [6]:
# map and explore damaturu_idp
damaturu_idp.explore()

In [7]:
# gain insight into damaturu_idp
print(damaturu_idp.head())
print(damaturu_idp.columns)
print(damaturu_idp[['Description']])


                  Name                                        Description  \
293   MOH'D GOMBE FARM  <html xmlns:fo="http://www.w3.org/1999/XSL/For...   
289   BUKAR ABBA HOUSE  <html xmlns:fo="http://www.w3.org/1999/XSL/For...   
288        ABARI Y.B.C  <html xmlns:fo="http://www.w3.org/1999/XSL/For...   
292  HURI VILLAGE CAMP  <html xmlns:fo="http://www.w3.org/1999/XSL/For...   

                                geometry  
293  POINT Z (12.03584 11.72675 0.00000)  
289  POINT Z (11.98183 11.72985 0.00000)  
288  POINT Z (11.98050 11.72993 0.00000)  
292  POINT Z (12.01215 11.73905 0.00000)  
Index(['Name', 'Description', 'geometry'], dtype='object')
                                           Description
293  <html xmlns:fo="http://www.w3.org/1999/XSL/For...
289  <html xmlns:fo="http://www.w3.org/1999/XSL/For...
288  <html xmlns:fo="http://www.w3.org/1999/XSL/For...
292  <html xmlns:fo="http://www.w3.org/1999/XSL/For...


In [20]:
# damaturu_idp shapefile to dataframe for easy data analysis
damaturu_idp_df = transform_idp_description(damaturu_idp)

In [ ]:
# save damaturu_idp_df to csv
damaturu_idp_df.to_csv("data/interim/damaturu_idp_df.csv", index=False)

In [15]:
# damaturu_idp shapefile to clean geojson with all attributes as columns for easy mapping
damaturu_idp_clean = transform_idp_complete(damaturu_idp)



In [16]:
# gain insight into damaturu_idp_clean and look at what columns we have
print("Columns in transformed data:")
print(damaturu_idp_clean.columns)

# gain insight into damaturu_idp_clean and look at first row
print("\nFirst site data:")
print(damaturu_idp_clean.iloc[0])

Columns in transformed data:
Index(['name', 'dtm_round', 'date', 'state', 'state_pcode', 'lga', 'ward',
       'siteid', 'site_name', 'longitude',
       ...
       'percent_of_children_attending_school',
       'occupation_of_majority_of_idps', '#_of_hh_with_source_of_income',
       'access_to_income_generating_activities', 'livestock_on_site',
       'idps_have_access_to_land_for_cultivation',
       'most_trusted_source_of_info',
       'main_topic_community_is_requesting_info',
       'serious_problem_due_to_lack_information', 'geometry'],
      dtype='object', length=102)

First site data:
name                                                     MOH'D GOMBE FARM
dtm_round                                                             R39
date                                                           06/10/2021
state                                                                YOBE
state_pcode                                                         NG036
                           

###### 3.2 Check Shapefile Data CRS

In [14]:
# save damaturu_idp_clean to geojson for mapping
damaturu_idp_clean.to_file('data/processed/damaturu_idp_clean.geojson', driver='GeoJSON')